**Modelo lineal para predecir mortalidad**

## CARGA DE LIBRERÍAS

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from functools import reduce

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

import plotly.express as px
import plotly.graph_objects as go
from IPython.display import Markdown


In [2]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


## CARGA DE DATOS Y PRIMER FILTRADO
* Filtrar según rangos de edad. Algunas gráficas,

In [3]:
path = '/content/drive/MyDrive/TFM/nuevosdatos'
df_excel_01  = pd.read_excel(path+'/EXCEL_01.xlsx')

/usr/local/lib/python3.7/dist-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [4]:
my_codes = [39075, 39085, 39035, 48020, 33024, 24089, 1059, 37274] # Seleccionar algunos municipios
my_prefixes = ['TC'] # Filtrar columnas

In [5]:
 # Seleccionar algunos municipios
#temp = df_excel_01[df_excel_01["CODMUN"].isin(my_codes)]
temp = df_excel_01
# Filtar columnas
df_info_population = temp[temp.columns[~pd.Series(temp.columns).str.startswith(tuple(my_prefixes))]]

In [6]:
df_excel_01b  = pd.read_excel(path+'/EXCEL_01B.xlsx')

In [7]:
my_prefixes = ['NOMBRE', "HTC", "CODMUN.1", "NOMBRE.1", "MTC"] # Filtrar columnas
# Seleccionar algunos municipios
#temp = df_excel_01b[df_excel_01b["CODMUN"].isin(my_codes)]
temp = df_excel_01b
# Filtar columnas
df_man_women = temp[temp.columns[~pd.Series(temp.columns).str.startswith(tuple(my_prefixes))]]

In [8]:
df_excel_02  = pd.read_excel(path+'/EXCEL_02.xlsx')

In [9]:
my_prefixes = ["TBNAT.1", "TBMOR.1", "Provincia", "Municipio"] # Filtrar columnas
# Seleccionar algunos municipios
#temp = df_excel_02[df_excel_02["CODMUN"].isin(my_codes)]
temp = df_excel_02
# Filtar columnas
df_nat_mor = temp[temp.columns[~pd.Series(temp.columns).str.startswith(tuple(my_prefixes))]]

In [10]:
df_excel_03  = pd.read_excel(path+'/EXCEL_03.xlsx')

/usr/local/lib/python3.7/dist-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [11]:
my_prefixes = ["SALDOTT", "CODMUN"] # Filtrar columnas
# Seleccionar algunos municipios
#temp = df_excel_03[df_excel_03["CODMUN"].isin(my_codes)]
temp = df_excel_03
# Filtar columnas
df_saldo = temp[temp.columns[pd.Series(temp.columns).str.startswith(tuple(my_prefixes))]]

In [12]:
df_excel_04  = pd.read_excel(path+'/EXCEL_04.xlsx')

/usr/local/lib/python3.7/dist-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [13]:
my_prefixes = ["POB", "NOMBRE", "PROVINCIA", "CCAA", "NATCODE"] # Filtrar columnas
# Seleccionar algunos municipios
#temp = df_excel_04[df_excel_04["CODMUN"].isin(my_codes)]
temp = df_excel_04
# Filtar columnas
df_ages_data = temp[temp.columns[~pd.Series(temp.columns).str.startswith(tuple(my_prefixes))]]

In [14]:
dfs = [df_info_population, df_man_women, df_nat_mor, df_ages_data]
df_final = reduce(lambda left,right: pd.merge(left,right,on='CODMUN'), dfs)

In [15]:
cols_identifiers = df_final.columns[:9]

In [16]:
men_2021 = []
men_2010 = []
men_2003 = []
for elem in df_final.columns:
  if elem[0] == "H":
    if (elem[0:2] not in  ["HO", "HP", "HI"]):
      if elem.split("_")[1] == "10":
          men_2010.append(elem)
      if elem.split("_")[1] == "03":
          men_2003.append(elem)
      if elem.split("_")[1] == "21":
          men_2021.append(elem)

In [17]:
women_2021 = []
women_2010 = []
women_2003 = []
for elem in df_final.columns:
  if elem[0] == "M":
    if (elem[0:2] not in  ["MO", "MU", "MP", "ME"]):
      if elem.split("_")[1] == "10":
          women_2010.append(elem)
      if elem.split("_")[1] == "03":
          women_2003.append(elem)
      if elem.split("_")[1] == "21":
          women_2021.append(elem)

In [18]:
temp = df_final[[*["CODMUN", "Nombre"], *men_2010]]
df_selected = temp[temp["CODMUN"].isin(my_codes)].\
       melt(id_vars = ["CODMUN", "Nombre"], var_name = "Rango", value_name = "Total_Rango")
       #set_index(["CODMUN"]).\
       #sort_index()
                                                 

In [19]:
fig = px.line(df_selected, 
        x="Rango", 
        y="Total_Rango", 
        color="Nombre", 
        title="Población masculina por rango de edad y municipio en 2010",
        labels = {"Nombre" : "Municipio",
                  "CODMUN" : "Código"},
        custom_data = ["CODMUN"])

for elem in fig.data:
  temp = elem.hovertemplate
  elem.hovertemplate = temp + '<br>' + "Nombre=%{customdata[0]}" + '<br>' + "Codigo Municipio=%{customdata[1]}"


fig.show()


## MODELO LINEAL PARA LA MORTALIDAD 1: 
Se entrena y comprueba con los datos disponibles para 2003 y 2010. Las variables predictoras relativas al número de personas por rango de edad sólo están disponible para 2003 y 2010. 

* Variable objetivo: mortalidad total
* Variables explicativas: número de personas por rango de edad, población total masculina, población total femenina.


### PARTE 1:
**PORBLEMA DE ESTE MODELO: SÓLO HAY DOS OBSERVACIONES POR MUNICIPIO. PERO HAY MUCHOS MUNICIPIOS...** 

**Posible solución:** ¿modelo que incluya otras variables predictoras?


### PARTE 2:
Entrenar con todos los datos de 2003 y 2010. Predecir para 2021

In [20]:
x_1 = df_final[ [*["CODMUN", "POB2003_x", "HOM2003_x", "MUJ2003_x"], *men_2003, *women_2003]]

d_2003 = {}
for elem in x_1.columns:
  if elem[0:3] == "POB" or elem[0:3] == "HOM" or elem[0:3] == "MUJ" or elem[0:3] == "MOR":
    temp = elem[0:3]
  else:
    temp = elem.split("_")[0]
  d_2003[elem] = temp

x_1 = x_1.rename(columns = d_2003)

In [21]:
y_1 = df_final[["CODMUN", "MOR2003"]].\
  rename(columns = {"MOR2003" : "MOR"})

In [22]:
df_1 = reduce(lambda left,right: pd.merge(left,right,on='CODMUN'), [x_1, y_1])\
        .dropna()

In [23]:
x_2 = df_final[ [*["CODMUN", "POB2010_x", "HOM2010_x", "MUJ2010_x"], *men_2010, *women_2010]]

d_2010 = {}
for elem in x_2.columns:
  if elem[0:3] == "POB" or elem[0:3] == "HOM" or elem[0:3] == "MUJ" or elem[0:3] == "MOR":
    temp = elem[0:3]
  else:
    temp = elem.split("_")[0]
  d_2010[elem] = temp

x_2 = x_2.rename(columns = d_2010)

In [24]:
y_2 = df_final[["CODMUN", "MOR2010"]].\
  rename(columns = {"MOR2010" : "MOR"}).\
  dropna()

In [25]:
df_2 = reduce(lambda left,right: pd.merge(left,right,on='CODMUN'), [x_2, y_2]).\
      dropna()

In [26]:
df_model1 = pd.concat([df_1, df_2])

In [27]:
x = df_model1[set(df_model1.columns) - set(["CODMUN", "MOR"])]
y = df_model1["MOR"]

In [28]:
# Trian - test split
x_train, x_test,y_train,y_test = train_test_split(x,y,test_size =0.2)

In [29]:
# Fit linear regression model
clf = LinearRegression().fit(x_train, y_train)
print('coefficient of determination:', clf.score(x_train, y_train))
print('intercept:', clf.intercept_)
print('slope:', clf.coef_)

coefficient of determination: 0.9973726240579855
intercept: -0.17452281929017488
slope: [ 7.60187345e+09  1.16951899e+08  1.16951899e+08  7.60187345e+09
  1.16951899e+08  1.16951899e+08  1.16951899e+08  1.16951899e+08
  7.60187345e+09  7.60187345e+09  1.16951899e+08  1.16951899e+08
  7.60187345e+09  7.60187345e+09  7.60187345e+09  7.60187345e+09
  7.60187345e+09  7.60187345e+09  7.60187345e+09  1.16951899e+08
  7.60187345e+09  1.16951899e+08  1.16951899e+08  1.16951899e+08
  2.45598989e+09  1.16951899e+08  1.16951899e+08  1.16951899e+08
  7.60187345e+09  7.60187345e+09  7.60187345e+09  7.60187345e+09
  7.60187345e+09  1.16951899e+08  7.60187345e+09  1.16951899e+08
  1.16951899e+08  1.16951899e+08  1.16951899e+08  7.60187345e+09
  1.16951899e+08 -2.57294178e+09 -5.02893167e+09  7.60187345e+09
  7.60187345e+09]


In [30]:
# predict on test data
y_pred = clf.predict(x_test)

In [31]:
# Compute MSE
mean_squared_error(y_test, y_pred)

393.6234200930171

In [32]:
# Compute MAE
mean_absolute_error(y_test, y_pred)

3.6226360702764

In [33]:
# Compute R2
r2_score(y_test, y_pred)

0.9945043567928081

Seguiremos esta idea en cuanto a variables predictoras y variable objetivo.

Entrenamos el modelo ocn TODOS los datos disponibles para 2003 y 2010, años de los cuales también conocemos la mortalidad.

Además, conocemos las variables predictoras para año 2021 pero no la variable mortalidad para el año 2021. Predeciremos con el modelo en estos datos de 2021. Si bien, no podremos evaluar los resultados, pero, hemos comprobado que le modelo es más o menos aceptable.

¿Sería interesante meter como predictora alguna característica propia del municipio (altura,....)?

In [34]:
x_2021 = df_final[ [*["CODMUN", "POB2021", "HOM2021_x", "MUJ2021_x"], *men_2021, *women_2021]]

d_2021 = {}
for elem in x_1.columns:
  if elem[0:3] == "POB" or elem[0:3] == "HOM" or elem[0:3] == "MUJ" or elem[0:3] == "MOR":
    temp = elem[0:3]
  else:
    temp = elem.split("_")[0]
  d_2021[elem] = temp

x_2021 = x_1.rename(columns = d_2021).\
            drop(["CODMUN"], axis = 1).\
            dropna()
x_2021['HOM'] = pd.to_numeric(x_2021['HOM'], errors = "coerce")
x_2021['MUJ'] = pd.to_numeric(x_2021['MUJ'], errors = "coerce")

In [35]:
# Fit linear regression model
clf3 = LinearRegression().fit(x, y)
print('coefficient of determination:', clf3.score(x, y))
print('intercept:', clf3.intercept_)
print('slope:', clf3.coef_)

coefficient of determination: 0.9982946252877503
intercept: -0.13518953688306468
slope: [-6.80226867e+08 -1.04650287e+07 -1.04650287e+07 -6.80226867e+08
 -1.04650288e+07 -1.04650286e+07 -1.04650286e+07 -1.04650287e+07
 -6.80226867e+08 -6.80226867e+08 -1.04650287e+07 -1.04650287e+07
 -6.80226867e+08 -6.80226867e+08 -6.80226866e+08 -6.80226867e+08
 -6.80226867e+08 -6.80226867e+08 -6.80226867e+08 -1.04650287e+07
 -6.80226867e+08 -1.04650287e+07 -1.04650287e+07 -1.04650286e+07
 -2.19765603e+08 -1.04650287e+07 -1.04650292e+07 -1.04650285e+07
 -6.80226867e+08 -6.80226867e+08 -6.80226867e+08 -6.80226867e+08
 -6.80226867e+08 -1.04650286e+07 -6.80226867e+08 -1.04650286e+07
 -1.04650285e+07 -1.04650284e+07 -1.04650286e+07 -6.80226867e+08
 -1.04650298e+07  2.30230632e+08  4.49996235e+08 -6.80226867e+08
 -6.80226867e+08]


In [36]:
y_pred_2021 = clf3.predict(x_2021)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:493: FutureWarning:

The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.




## MODELO LINEAL PARA LA MORTALIDAD 2
* Variable objetivo: mortalidad total 
* Variables predictoras: poblacion masculina, poblacion femenina, natalidad, tasa bruta natalidad, tasa bruta mortalidad, saldo vegetativo, tasa crecimiento interanual

**Este modelo no tiene mucho sentido, pues, de antemano, conoce variables equivalentes a la moratlidad. Además, en el modelo de gemelos digitales, no va a conocerlas; probablemente sólo conozca como predictora sen total de población según sexo, según edad;  el total de población masculina, el totlal de población femenina.** 

In [37]:
# Select desired columns and distinguish by year
def build_up_dfs(prefixes_list):
  df_list  =[]
  for prefix in prefixes_list:
    cols = [elem for elem in df_final.columns if elem.startswith(prefix) 
            and (elem[-1] not in ['x', 'y']) and (elem[3] != "1")]
    # Save in dataframe
    df_temp = df_final[[*["CODMUN"], *cols]].\
             melt(id_vars = ["CODMUN"], var_name = "Year", value_name = prefix)
    # Change name; just want the year
    df_temp["Year"] = list(map(lambda x: x[-4:], df_temp["Year"]))
    df_list.append(df_temp)
  return(df_list)

In [38]:
prefixes = ["HOM", "MUJ", "SVEG", "NAT", "TBNAT", "TBMOR", "TCVEG", "MOR"]
df_list = build_up_dfs(prefixes)

In [39]:
df_model2 = reduce(lambda  left,right: pd.merge(left,right,on=["CODMUN", "Year"],
                                                   how='outer'), df_list)
df_model2['HOM'] = pd.to_numeric(df_model2['HOM'], errors = "coerce")
df_model2['MUJ'] = pd.to_numeric(df_model2['MUJ'], errors = "coerce")

df_model2 = df_model2.dropna()

In [40]:
# Predictors
x = df_model2[["HOM", "MUJ", "SVEG", "NAT", "TBNAT", "TBMOR", "TCVEG"]]
# Target variable / predictand
y = df_model2["MOR"]

In [41]:
x.head()

,HOM,MUJ,SVEG,NAT,TBNAT,TBMOR,TCVEG
0,799.0,816.0,-22,6.0,32.352941,82.352941,-64.705882
1,5141.0,5272.0,-158,55.0,321.888412,914.163090,-678.111588
2,362.0,266.0,-14,1.0,9.433962,141.509434,-132.075472
3,36800.0,40991.0,-680,384.0,2600.877193,4666.666667,-2982.456140
4,803.0,876.0,-24,4.0,85.714286,266.666667,-228.571429


In [42]:
# BUILD UP MODEL
# train - test splitting
x_train, x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2)
# Train Linear Regression model
clf2 = LinearRegression().fit(x_train, y_train)
print('coefficient of determination:', clf2.score(x_train, y_train))
print('intercept:', clf2.intercept_)
print('slope:', clf2.coef_)

coefficient of determination: 1.0
intercept: -7.105427357601002e-15
slope: [-7.37396709e-17  5.55111512e-17 -1.00000000e+00  1.00000000e+00
 -1.50162001e-16  1.94072189e-17 -9.81202966e-17]


In [43]:
y_pred = clf2.predict(x_test)

In [44]:
# MSE
mean_squared_error(y_test, y_pred)

1.1911926460075424e-26

In [45]:
# MAE
mean_absolute_error(y_test, y_pred)

2.0503014301223225e-14

In [46]:
# R2
r2_score(y_test, y_pred)

1.0